# Collecting cities info - Web Scraping

In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from lat_lon_parser import parse
import datetime
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pytz import timezone
from datetime import datetime


## Creating the function 

In [2]:
def get_cities_info(cities):
    add_data = []  

    for city in cities:
        url = f'https://en.wikipedia.org/wiki/{city}'
        headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
                    }
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Error fetching data for {city}: {response.status_code}")
            continue  

        soup = BeautifulSoup(response.content, 'html.parser')

        lat = soup.find(class_='latitude')
        lon = soup.find(class_='longitude')
        country = soup.find('th', string='Country')
        pop = soup.find(string='Population').find_next(string=re.compile(r'\d+,\d+,\d+'))

        if not (lat and lon and country and pop):
            print(f"Skipping {city}, missing some data.")
            continue  

        lat = parse(lat.text)
        lon = parse(lon.text)  
        country = country.find_next('td').text.strip()
     
        add_data.append({
            "city_name": city,
            "country_name": country,
            "longitude": lon,
            "latitude": lat,
            "population": pop,
            "year_data_retrieved": datetime.now()
        })

    return pd.DataFrame(add_data)  

In [3]:
get_cities_info(['Berlin','Hamburg', 'Munich'])

,city_name,country_name,longitude,latitude,population,year_data_retrieved
0,Berlin,Germany,13.405,52.5200,"3,596,999",2025-10-15 16:12:05.554175
1,Hamburg,Germany,10.000,53.5500,"1,964,021",2025-10-15 16:12:06.486546
2,Munich,Germany,11.575,48.1375,"1,505,005",2025-10-15 16:12:07.620851


## Constructing dataframes: connecting phyton with my SQL

In [44]:
load_dotenv()

True

In [45]:
password_mysql=os.getenv('mysql_password')
my_user_mysql=os.getenv('username')
my_sql_port=os.getenv('my_port')
my_localhost=os.getenv('localhost')

In [46]:
schema = "gans"
host = my_localhost
user = my_user_mysql
password = password_mysql
port = my_sql_port

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


#### cities_df

In [6]:
cities_info_df= pd.DataFrame(get_cities_info(['Berlin','Hamburg', 'Munich']))
cities_info_df

,city_name,country_name,longitude,latitude,population,year_data_retrieved
0,Berlin,Germany,13.405,52.5200,"3,596,999",2025-10-15 16:13:36.504072
1,Hamburg,Germany,10.000,53.5500,"1,964,021",2025-10-15 16:13:38.123471
2,Munich,Germany,11.575,48.1375,"1,505,005",2025-10-15 16:13:39.409996


In [7]:
cities_df = pd.DataFrame({
    "city_name": cities_info_df["city_name"].values,
    "country": cities_info_df["country_name"].values
})
cities_df


,city_name,country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


> **Important!**
> Fill Gans Database to use the city_id primary key

In [ ]:
engine = create_engine(connection_string)

cities_df.to_sql('cities', 
                 if_exists='append', 
                 con=engine,
                 index=False)

3

In [47]:
# To confirm!
cities_df_from_sql = pd.read_sql("cities", con=connection_string)
cities_df_from_sql

,city_id,city_name,country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany


#### additional_data_df

In [20]:
add_data_df = (cities_info_df
               .merge(cities_df_from_sql, on="city_name", how="left")
               .drop(columns=["city_name", "country_name", "country"]))

In [21]:
add_data_df.to_sql('additional_data',
                if_exists='append',
                con=connection_string,
                index=False)

3

In [22]:
add_data_df_from_sql = pd.read_sql("additional_data", con=connection_string)
add_data_df_from_sql

,city_id,population,longitude,latitude,year_data_retrieved
0,1,"3,596,999",13.0,53.0,2025-10-15 16:13:37
1,2,"1,964,021",10.0,54.0,2025-10-15 16:13:38
2,3,"1,505,005",12.0,48.0,2025-10-15 16:13:39
